# Project 2

You have been approached by The Director of Yemisi Shyllon Museum of Art (YSMA) to build a computer vision program with a graphical user interface (GUI) that would ask for a username and password from visitors to log in and view and perform image enhancements on the art collection at the museum. Given the knowledge of the collection categories from the previous project.

Hint: Use the Python Tinker library to build your GUI


In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from PIL import ImageTk, Image

In [2]:
def translate_image(image_path, dx, dy):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    translated_img = cv2.warpAffine(img, M, (cols, rows))
    
    return img, translated_img
    
def reflect_image(image_path):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = np.float32([[1, 0, 0], [0, -1, rows]])
    reflected_img = cv2.warpAffine(img, M, (cols, rows))

    return img, reflected_img

def rotate_image(image_path, angle, scale):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, scale)
    rotated_img = cv2.warpAffine(img, M, (cols, rows))

    return img, rotated_img

def crop_image(image_path, x1, y1, x2, y2):
    img = cv2.imread(image_path, 0)
    cropped_img = img[y1:y2, x1:x2]
    
    return img, cropped_img

def shear_image_x(image_path, shear_factor):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = np.float32([[1, shear_factor, 0], [0, 1, 0], [0, 0, 1]])
    sheared_img = cv2.warpPerspective(img, M, (int(cols*1.5), int(rows*1.5)))
    
    return img, sheared_img

def shear_image_y(image_path, shear_factor):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = np.float32([[1, 0, 0], [shear_factor, 1, 0], [0, 0, 1]])
    sheared_img = cv2.warpPerspective(img, M, (int(cols*1.5), int(rows*1.5)))
    
    return img, sheared_img

def apply_blur(image_path, blur_type):
    img = cv2.imread(image_path)
    
    if blur_type == 'Gaussian Blur':
        blurred_img = cv2.GaussianBlur(img, (7, 7), 0)
    elif blur_type == 'Median Blur':
        blurred_img = cv2.medianBlur(img, 5)
    elif blur_type == 'Bilateral Blur':
        blurred_img = cv2.bilateralFilter(img, 9, 75, 75)
    else:
        print("Invalid blur type.")
        return
    return img, blurred_img



In [3]:
def login(root, username_entry, password_entry):
    username = username_entry.get()
    password = password_entry.get()

    if username == "admin" and password == "password":
        messagebox.showinfo("Login Successful", "Welcome, Admin!")
        root.destroy()  
        open_transformation_window()  
    else:
        messagebox.showerror("Login Failed", "Invalid username or password.")


In [4]:
def open_image(original_image_label):
    global selected_image_path
    selected_image_path = None
    
    filename = filedialog.askopenfilename()
    if filename:
        selected_image_path = filename
        
        img = cv2.imread(filename)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img = Image.fromarray(img)
        img.thumbnail((300, 400))
        img = ImageTk.PhotoImage(img)
        
        original_image_label.config(image=img)
        original_image_label.image = img

        
# Center a window on the screen.
def center_window(window):
    window.update_idletasks()
    
    width = window.winfo_width()
    height = window.winfo_height()
    
    x_offset = (window.winfo_screenwidth() - width) // 2
    y_offset = (window.winfo_screenheight() - height) // 2
    
    window.geometry(f"{width}x{height}+{x_offset}+{y_offset}")

#  Display the transformed image in the transformation window.
def display_transformed_image(transformed_img, transformation_parameters_window, transformed_image_label):

    # Check if the image is grayscale
    is_grayscale = len(transformed_img.shape) == 2 or transformed_img.shape[2] == 1

    # Convert BGR image to RGB for display if not grayscale
    if not is_grayscale:
        transformed_img_rgb = cv2.cvtColor(transformed_img, cv2.COLOR_BGR2RGB)
    else:
        transformed_img_rgb = transformed_img

    # Convert NumPy array to PIL Image
    transformed_img_pil = Image.fromarray(transformed_img_rgb)

    # Resize the image to fit the label
    transformed_img_pil.thumbnail((300, 400))  

    # Convert PIL Image to PhotoImage
    transformed_img_tk = ImageTk.PhotoImage(transformed_img_pil)

    # Display the transformed image in the right-side frame
    transformed_image_label.config(text="Transformed Image")
    transformed_image_label.config(image=transformed_img_tk)
    transformed_image_label.image = transformed_img_tk

    # Pack the label to update the display
    transformed_image_label.pack(expand=True, fill="both")

    # Destroy the transformation window
    transformation_parameters_window.destroy()

In [5]:
# Define entry fields and other widgets globally
transformation_entries = {}

def open_transformation_parameters_window(transformation_window, transformed_image_label, transformation_dropdown):
    global transformation_entries
    
     # Clear the transformation_entries dictionary
    transformation_entries.clear()
    
    if selected_image_path is not None:

        # Create a new window for transformation parameters
        transformation_parameters_window = tk.Toplevel(transformation_window)
        transformation_parameters_window.title("Transformation Parameters")
        transformation_parameters_window.geometry("600x600")
        transformation_parameters_window.resizable(False, False)

        # Center the transformation parameters window on the screen
        center_window(transformation_parameters_window)

        selected_transformation = transformation_dropdown.get()

        # Define parameter labels and entry fields for each transformation
        parameters = {
            "Translate": ["dx", "dy"],
            "Reflect": [],
            "Rotate": ["Angle", "Scale"],
            "Shear X": ["Shear Factor"],
            "Shear Y": ["Shear Factor"],
            "Crop": ["x1", "y1", "x2", "y2"]
        }

        # Add widgets based on the selected transformation
        if selected_transformation in parameters:
            ttk.Label(transformation_parameters_window, text="Transformation Parameters").pack(pady=10)

            for param in parameters[selected_transformation]:
                ttk.Label(transformation_parameters_window, text=param + ":").pack(pady=5)
                entry = ttk.Entry(transformation_parameters_window)
                entry.pack(pady=5)
                transformation_entries[param] = entry

            # Button to apply transformation
            ttk.Button(transformation_parameters_window, text="Apply Transformation", command=lambda: perform_transformation(selected_transformation, transformation_parameters_window, transformed_image_label)).pack(pady=10)

        elif selected_transformation in ["Gaussian Blur", "Median Blur", "Bilateral Blur"]:
            ttk.Button(transformation_parameters_window, text="Apply Blur", command=lambda: perform_transformation(selected_transformation, transformation_parameters_window, transformed_image_label)).pack(pady=10)

        else:
            ttk.Label(transformation_parameters_window, text="Transformation not yet implemented").pack(pady=10)
    
    else:
        messagebox.showinfo("Info", "No image selected.")


In [6]:
def perform_transformation(selected_transformation, transformation_parameters_window, transformed_image_label):
    
    # Check if the selected transformation requires parameters
    transformations_with_params = ["Translate", "Rotate", "Shear X", "Shear Y", "Crop"]

    if selected_transformation in transformations_with_params and not transformation_entries:
        messagebox.showerror("Error", "No transformation parameters entered.")
        return


    # Get the selected transformation and its parameters
    parameters = {param: entry.get() for param, entry in transformation_entries.items()}

    # Apply the selected transformation based on its name
    if selected_transformation == "Translate":
        img, transformed_img = translate_image(selected_image_path, int(parameters["dx"]), int(parameters["dy"]))
    elif selected_transformation == "Reflect":
        img, transformed_img = reflect_image(selected_image_path)
    elif selected_transformation == "Rotate":
        img, transformed_img = rotate_image(selected_image_path, float(parameters["Angle"]), float(parameters["Scale"]))
    elif selected_transformation == "Shear X":
        img, transformed_img = shear_image_x(selected_image_path, float(parameters["Shear Factor"]))
    elif selected_transformation == "Shear Y":
        img, transformed_img = shear_image_y(selected_image_path, float(parameters["Shear Factor"]))
    elif selected_transformation == "Crop":
        img, transformed_img = crop_image(selected_image_path, int(parameters["x1"]), int(parameters["y1"]), int(parameters["x2"]), int(parameters["y2"]))
    elif selected_transformation in ["Gaussian Blur", "Median Blur", "Bilateral Blur"]:
        img, transformed_img = apply_blur(selected_image_path, selected_transformation)
    else:
        messagebox.showerror("Error", "Invalid transformation selected.")
        return

    # Display the transformed image
    if transformed_img is not None:
        display_transformed_image(transformed_img, transformation_parameters_window, transformed_image_label)

In [7]:
def open_transformation_window():
    transformation_window = tk.Tk()
    transformation_window.title("YSMA Image Transformation App")
    transformation_window.geometry("1200x600")
    transformation_window.resizable(False, False)

    # Center the transformation parameters window on the screen
    center_window(transformation_window)
    
    style = ttk.Style()
    style.theme_use("vista")
    style.configure("TLabel", padding=6)
    style.configure("TEntry", padding=6)
    style.configure("TButton", padding=6)

    # Create a frame for the left section (original image)
    left_frame = ttk.Frame(transformation_window, width=transformation_window.winfo_width() // 2, borderwidth=2, relief="solid")
    left_frame.pack(side="left", expand=True, fill="both")

    # Label for original image title
    original_image_title_label = ttk.Label(left_frame, text="Original Image", font=("Helvetica", 16, "bold"))
    original_image_title_label.pack(side="top", padx=10, pady=(10, 5))

    # Label for original image
    original_image_label = ttk.Label(left_frame, borderwidth=2, relief="solid")
    original_image_label.pack(expand=True, fill="both")

    # Button to select original image
    select_image_button = ttk.Button(left_frame, text="Select Original Image", command=lambda: open_image(original_image_label))
    select_image_button.pack(pady=30)

    # Create a frame for the right section (transformed image)
    right_frame = ttk.Frame(transformation_window, width=transformation_window.winfo_width() // 2, borderwidth=2, relief="solid")
    right_frame.pack(side="right", expand=True, fill="both")

    # Label for transformed image title
    transformed_image_title_label = ttk.Label(right_frame, text="Transformed Image", font=("Helvetica", 16, "bold"))
    transformed_image_title_label.pack(side="top", padx=10, pady=(10, 5))

    # Label for transformed image
    transformed_image_label = ttk.Label(right_frame, borderwidth=2, relief="solid")
    transformed_image_label.pack(expand=True, fill="both")

    # Label for selecting transformation
    transformation_label = ttk.Label(right_frame, text="Select Transformation:")
    transformation_label.pack(side="left", padx=(20, 10), pady=30, anchor="w")

    transformation_choices = ["Translate", "Reflect", "Rotate", "Crop", "Shear X", "Shear Y", "Gaussian Blur", "Median Blur", "Bilateral Blur"]
    transformation_dropdown = ttk.Combobox(right_frame, values=transformation_choices, state="readonly")
    transformation_dropdown.pack(side="left", padx=(0, 10), pady=5)

    # Button to open transformation window
    open_transformation_button = ttk.Button(right_frame, text="Open Transformation Window", command=lambda: open_transformation_parameters_window(transformation_window, transformed_image_label, transformation_dropdown))
    open_transformation_button.pack(side="right", padx=(10, 20), pady=30)


In [8]:
def ysma_image_transformation_app():
    root = tk.Tk()
    root.title("YSMA Image Transformation App | Login")
    root.geometry("400x200")

    center_window(root)
    root.resizable(False, False)

    # Apply a modern style
    style = ttk.Style()
    style.theme_use("vista")
    style.configure("TLabel", padding=6)
    style.configure("TEntry", padding=6)
    style.configure("TButton", padding=6)

    # Login Frame
    login_frame = ttk.Frame(root)
    login_frame.pack(expand=True, padx=20, pady=20)

    # Username Label and Entry
    username_label = ttk.Label(login_frame, text="Username:")
    username_label.grid(row=0, column=0, sticky=tk.W)

    username_entry = ttk.Entry(login_frame, width=30)
    username_entry.grid(row=0, column=1, sticky=tk.W, pady=5)

    # Password Label and Entry
    password_label = ttk.Label(login_frame, text="Password:")
    password_label.grid(row=1, column=0, sticky=tk.W)

    password_entry = ttk.Entry(login_frame, show="*", width=30)
    password_entry.grid(row=1, column=1, sticky=tk.W, pady=5)

    # Login Button
    login_button = ttk.Button(login_frame, text="Login", command= lambda:login(root, username_entry, password_entry), width=30)
    login_button.grid(row=2, column=0, columnspan=2, pady=(20, 0))

    # Start the Tkinter event loop
    root.mainloop()

In [9]:
ysma_image_transformation_app()